# Getting Clumps

I will be focusing on only getting clump spectra from SDSS here.

## Preliminary Functions

In [317]:
from astropy.io import fits
from astropy import table
from astroquery.sdss import SDSS
from astropy import coordinates as coords
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mechanize
from io import BytesIO
from astropy.table import Table
from astropy.table import vstack
from astropy import units as u
from astropy.io import ascii
import matplotlib as mpl
from tqdm import tqdm_notebook as tqdm

In [2]:
mpl.style.use('dark_background')
mpl.rcParams['figure.facecolor'] = 'k'

## Generating Metallicities

Let's use the M91 calibration mentioned in [Kewley & Ellison 2008 (KE08)](http://adsabs.harvard.edu/abs/2008ApJ...681.1183K):

$$12 + \log(O/H)_\text{lower} = 12 - 4.944 + 0.767x + 0.602x^2 - y(0.29+0.332x - 0.331x^2)$$

$$12 + \log(O/H)_\text{upper} = 12 - 2.939 - 0.2x - 0.37x^2 - 0.305 x^3 - 0.0283x^4 - y(0.0047 - 0.221x - 0.102x^2 - 0.0817x^3 - 0.00717x^4)$$

where $$x = \log R_{23} = \log\left[\frac{[\text{OII}\lambda3727 + [\text{OIII}]\lambda4959 + [\text{OIII}]\lambda5007}{\text{H}\beta}\right]$$
and $$y = \log\text{O}_{32} = \log\left[\frac{[\text{OIII}]\lambda4959 + [\text{OIII}]\lambda5007}{[\text{OII}]\lambda3727}\right]$$

In [3]:
def getlogR23(OII3727,OIII4959,OIII5007,HB):
    return np.log10((OII3727 + OIII4959 + OIII5007)/HB)

def getlogO32(OIII4959,OIII5007,OII3727):
    return np.log10((OIII4959 + OIII5007)/OII3727)

def getlogOHlow(x,y):
    return (-4.944 + 0.767*x + 0.602*x**2 - y*(0.29 + 0.332*x - 0.331*x**2))

def getlogOHup(x,y):
    return (-2.939 - 0.2*x - 0.37*x**2 - 0.305*x**3 - 0.0283*x**4 - 
            y*(0.0047 - 0.221*x - 0.102*x**2 - 0.0817*x**3 - 0.00717*x**4))

### M91 Branches

We need to determine if we are using the upper branch or the lower branch. To do this, we need to get $$\log\left(\frac{[\text{NII}]\lambda6584}{[\text{OII}]\lambda3727}\right)$$

In [4]:
def getlogNiiOii(NII6584,OII3727):
    return np.log10(NII6584/OII3727)

If this value is greater than $-1.2$, then we use the upper branch, while if it is smaller, we use the lower branch.

In [211]:
def M91cal(obj):
    
    x = getlogR23(obj['OII3726'],obj['OIII4959'],obj['OIII5007'],obj['HBeta'])
    y = getlogO32(obj['OIII4959'],obj['OIII5007'],obj['OII3726'])
    
    branchCheck = getlogNiiOii(obj['NII6584'],obj['OII3726'])
    
    if branchCheck > -1.2:
        return branchCheck, getlogOHup(x,y) + 12
    else:
        return branchCheck, getlogOHlow(x,y) + 12

## `astroquery` SQL search

In [202]:
def getSpecDataRADEC(RA,DEC):
    pos = coords.SkyCoord(RA,DEC,unit='deg')
    xid = SDSS.query_region(pos,spectro = True,radius = 20*u.arcsec)
    
    if not xid:
        return False, False
    
    if len(xid) > 1:
#         print ('oh no')
        xid = table.unique(xid,keys='objid',keep='last')
    
    sp = SDSS.get_spectra(matches=xid)
    spData = sp[0][1].data
    return xid

In [7]:
def genSQLsearch(xid):
    SQL = "SELECT \
    s.specObjID, g.class, g.subClass, g.z, g.ra, g.dec, g.plate, g.fiberID, g.MJD, \
    s.oii_3726_flux as OII3726, s.oiii_4959_flux as OIII4959,s.oiii_5007_flux as OIII5007, \
    s.h_beta_flux as HBeta,s.h_alpha_flux as HAlpha,s.nii_6584_flux as NII6584, \
    x.logMass as logM \
    FROM galSpecLine s \
    JOIN stellarMassFSPSGranEarlyDust x ON x.specObjID = s.specObjID \
    JOIN specObj g ON g.specObjID = s.specObjID \
    WHERE g.plate = %s AND g.fiberID = %s"%(xid['plate'],xid['fiberID'])
    return SQL

In [53]:
def genSQLsearchNoM(xid):
    SQL = "SELECT \
    s.specObjID, g.class, g.subClass, g.z, g.ra, g.dec, g.plate, g.fiberID, g.MJD, \
    s.oii_3726_flux as OII3726, s.oiii_4959_flux as OIII4959,s.oiii_5007_flux as OIII5007, \
    s.h_beta_flux as HBeta,s.h_alpha_flux as HAlpha,s.nii_6584_flux as NII6584 \
    FROM galSpecLine s \
    JOIN specObj g ON g.specObjID = s.specObjID \
    WHERE g.plate = %s AND g.fiberID = %s"%(xid['plate'],xid['fiberID'])
    return SQL

## Generating Tables

In [304]:
def tableGen(RA,DEC):
    xid = getSpecDataRADEC(RA,DEC)
    
    xidList = []
    for x in xid:
        if not x:
            print('No xid')
            continue
        xidList.append(x)
    
    if not xidList:
        return
    
    queries = [genSQLsearch(x) for x in xidList]
    noMqueries = [genSQLsearchNoM(x) for x in xidList]
    results = [SDSS.query_sql(query) for query in queries]
    resultsnoM = [SDSS.query_sql(query) for query in noMqueries]
    newResults = []
    for n in range(0,len(resultsnoM)):
        if resultsnoM[n] == None:
            continue
        elif None in resultsnoM[n]:
            continue
        resultsnoM[n]['subClass'] = resultsnoM[n]['subClass'].astype(str)
        resultsnoM[n]['logNiiOii'],resultsnoM[n]['M91'] = M91cal(resultsnoM[n])
        try:
            resultsnoM[n]['logM'] = results[n]['logM']
        except TypeError:
            resultsnoM[n]['logM'] = 0 
        newResults.append(resultsnoM[n])
    if len(newResults) > 0:
        rTable = vstack([res for res in newResults])
    else:
        return
    return rTable

### Testing

In [286]:
xid = getSpecDataRADEC(340.309, -0.621)
xid

ra,dec,objid,run,rerun,camcol,field,z,plate,mjd,fiberID,specobjid,run2d,instrument
float64,float64,int64,int64,int64,int64,int64,float64,int64,int64,int64,int64,bytes6,bytes4
340.30886495574,-0.620854386728136,1237663542613639326,4207,301,2,240,0.005918281,1101,52621,62,1239632883837069312,26,SDSS
340.310156254532,-0.617873852786732,1237663542613639327,4207,301,2,240,0.005904465,4204,55470,126,4733317934754754560,v5_7_0,BOSS


In [275]:
queries = [genSQLsearch(x) for x in xid]
noMqueries = [genSQLsearchNoM(x) for x in xid]
results = [SDSS.query_sql(query) for query in queries]
resultsnoM = [SDSS.query_sql(query) for query in noMqueries]

In [276]:
resultsnoM

[None, None]

In [296]:
if None in resultsnoM:
    print('ohno')

ohno


In [278]:
newResults = []
for n in range(0,len(resultsnoM)):
    if resultsnoM[n] == None:
        continue
    resultsnoM[n]['subClass'] = resultsnoM[n]['subClass'].astype(str)
    resultsnoM[n]['logNiiOii'],resultsnoM[n]['M91'] = M91cal(resultsnoM[n])
    try:
        resultsnoM[n]['logM'] = results[n]['logM']
    except TypeError:
        resultsnoM[n]['logM'] = 0 
    newResults.append(resultsnoM[n])

In [280]:
rTable = vstack([res for res in newResults])

ValueError: no values provided to stack.

In [262]:
rTable

specObjID,class,subClass,z,ra,dec,plate,fiberID,MJD,OII3726,OIII4959,OIII5007,HBeta,HAlpha,NII6584,logNiiOii,M91,logM
int64,bytes6,str9,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64
442554559789819904,GALAXY,False,0.01330698,10.349474,-0.064968354,393,276,51794,0,0.3852063,10.67733,11.60589,30.77504,5.468029,inf,-inf,9.23184975868996
1683282807406749696,GALAXY,STARBURST,0.01338809,10.346346,-0.063367,1495,227,52944,0,16.07723,55.49376,32.06713,80.72696,7.343747,inf,inf,6.90638247831332


### Interesting Objects:

In [263]:
intObjs = [(23.422,-1.158),(22.515,0.743),(4.878,-0.602),(37.884,-0.728),(37.861,-0.793),(329.604,-0.740),(10.349,-0.065),(13.241,0.369),
           (7.410,0.410),(11.389,-1.106),(35.901,0.226),(350.466,-0.691),(350.533,-0.054),(20.575,1.007),(46.347,-0.242),(323.217,-0.311),
           (41.796,-1.044),(18.415,0.875),(14.731,1.005),(23.281,0.176),(28.497,-0.750),(33.000,1.218),(57.287,1.163),(47.617,1.098),
           (43.355,-0.232),(41.489,-0.753),(357.970,0.755),(20.339,0.090),(7.948,0.559),(324.986,0.323),(2.526,-0.438),(32.201,0.800),
           (56.529,0.390),(2.915,-0.477),(34.749,-0.601),(45.264,-0.743),(312.717,-0.326),(340.309,-0.621),(333.085,0.561),(351.096,-0.108),
           (47.492,-0.694),(49.025,-0.440),(40.200,1.105),(7.437,0.170),(350.147,-0.881),(35.624,0.383),(47.889,-1.244),(43.711,1.057),(49.334,-0.077)]

In [325]:
not tableGen(*intObjs[37])

True

In [323]:
tableList = []
intBar = tqdm(total=len(intObjs))
for obj in intObjs:
    intBar.set_description(str(obj))
    tab = tableGen(*obj)
    if not tab:
        continue
    else:
        tableList.append(tab)
    intBar.update(1)

/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in true_divide
  """
/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  
/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log10
  
/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in log10
  """
/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [326]:
len(tableList)

48

In [328]:
tableList2 = []
for tab in tableList:
    if np.inf not in tab['logNiiOii']:
        tableList2.append(tab)

In [330]:
len(tableList2)

26

In [331]:
tableList2[0]

specObjID,class,subClass,z,ra,dec,plate,fiberID,MJD,OII3726,OIII4959,OIII5007,HBeta,HAlpha,NII6584,logNiiOii,M91,logM
int64,bytes6,str11,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
437980866312955904,GALAXY,STARBURST,0.03285039,4.8778482,-0.60175283,389,21,51795,326.444,120.0465,338.3001,235.3608,655.7142,132.9357,-0.3901670639614821,8.831685537003343,0.0
1678727256727382016,GALAXY,STARFORMING,0.03292816,4.879452,-0.60177,1491,38,52996,55.34501,9.218669,32.70643,48.17247,156.0314,34.77625,-0.20179572099619789,8.947588816527292,7.90438092122952
